In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier # wrapper to use function from sklearn
from tensorflow.keras import backend as be
from tensorflow.keras import layers, Sequential
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV #training and testing data split
from sklearn.preprocessing import StandardScaler

# import plotting libraries
import matplotlib.pyplot as plt
#import sns for better plots, it is handy to manage subplots
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import os
for dirname, _, filenames in os.walk('D:\Projects\AI Workgroup\MNIST\data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"



D:\Projects\AI Workgroup\MNIST\data\train.csv


In [5]:
data = pd.read_csv("D:/Projects/AI Workgroup/MNIST/data/train.csv")
data.head(5)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
      tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.15)
  except RuntimeError as e:
    print(e)
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session());

In [11]:
print(data.head())
print(data.shape)

train, test= train_test_split(data, test_size=0.15, stratify=data["label"])

Xtrain = train[1:]
ytrain = train[:1]

Xtest = test[1:]
ytest = test[:1]

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

In [12]:
window_length = 8 # 8 pixels -> 98 features left?784/8
window_length2 = 7 # 98 features -> 98/7 = 14 features

def create_model(optimizer, learning_rate, first_layer_filters, second_layer_filters):
    if optimizer=='SGD':
        optimizer = SGD(learning_rate = learning_rate)
    if optimizer=='RMSprop':
        optimizer = RMSprop(learning_rate = learning_rate)
    if optimizer=='Adagrad':
        optimizer = Adagrad(learning_rate = learning_rate)
    if optimizer=='Adadelta':
        optimizer = Adadelta(learning_rate = learning_rate)
    if optimizer=='Adam':
        optimizer = Adam(learning_rate = learning_rate)
    if optimizer=='Adamax':
        optimizer = Adamax(learning_rate = learning_rate)
    if optimizer=='Nadam':
        optimizer = Nadam(learning_rate = learning_rate)
    
    model = Sequential()
    model.add(layers.Conv1D(first_layer_filters, window_length, activation = 'relu', input_shape = (1,784)))
    model.add(layers.Conv1D(second_layer_filters, window_length2, activation = 'relu', input_shape = (1,98)))
    model.add(layers.Dense(14, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    
    #model.add(layers.Dropout(0.2))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [13]:
opti_model = create_model('Adam', 0.04, 98, 14)
opti_model.fit(Xtrain, ytrain, batch_size=92, epochs=5, shuffle=True)

ValueError: Negative dimension size caused by subtracting 8 from 1 for '{{node conv1d_1/conv1d}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d_1/conv1d/ExpandDims, conv1d_1/conv1d/ExpandDims_1)' with input shapes: [?,1,1,784], [1,8,784,98].